# Spider-Gender

## Fasi:
1. Estrazione dei dataset
2. Combinazione dei dataset

# Estrazione dei dataset
In questa fase vengono create le directory contenenti i dataset estratti.  
É necessario che nella cartella contenente i file si trovino i **.zip** dei 2 dataset.

In [56]:
#Inserire in una cartella di lavoro i file zip dei dataset
#Creazione delle directory ed estrazione dei dataset
import os
import utils as u

path=r"C:\Users\vince\Desktop\Progetto FVAB"
os.chdir(path)
cwd = os.getcwd()

#Crea directory per estrarre i dataset solo una volta all'inizio
u.makedir('celeba')
u.makedir('utkface')

#Estrae i due dataset solo una volta all'inizio
u.unzip('celeba-dataset.zip','celeba')
u.unzip('utkface-new.zip','utkface')

# Combinazione dei dataset
In questa fase sono combinati i due dataset e viene creato un indice che tiene traccia dei nuovi nomi 

| Old_Name    | New_Name   |
| :---: | :---: |
|   000001.jpg    | 000001_1.jpg     |
|  1_0_0_20161219140627985.jpg.chip.jpg  | 000002_0.jpg       |

Il nuovo nome è composto da:
* Parte sinistra un intero che incrementa di 1
* Parte destra dopo l'underscore che indica la label 
 0. Donna 
 1. Uomo

In [57]:
import utils as u
import os
path=r"C:\Users\vince\Desktop\Progetto FVAB"
os.chdir(path)

u.makedir('merged_dataset')
os.chdir('merged_dataset')
u.makedir('images')


path_images_celeba=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\img_align_celeba\img_align_celeba"
path_images_celeba2=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\img_align_celeba\img_align_celeba2"

path_images_utkface=r"C:\Users\vince\Desktop\Progetto FVAB\utkface\UTKFace"
path_images_utkface2=r"C:\Users\vince\Desktop\Progetto FVAB\\utkface\UTKFace2"

path_celeba_csv=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\list_attr_celeba.csv"
path_celeba_csv2=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\list_attr_celeba2.csv"

dest_path=r"C:\Users\vince\Desktop\Progetto FVAB\merged_dataset\images"
dest_path_oldnew_csv=r"C:\Users\vince\Desktop\Progetto FVAB\merged_dataset"

u.merge_datasets(path_images_celeba2,
                 path_images_utkface2,
                 path_celeba_csv2,
                 dest_path,
                 dest_path_oldnew_csv)
#u.merge_utk_only(path_images_utkface2,dest_path_oldnew_csv,dest_path)

# Estrazione dei Landmarks
In questa fase utilizzo l'algoritmo fornito per ottenere i landmarks sul volto  

<img style="float:left;" src="web.PNG">

In [58]:
%%capture
os.chdir(path)
import calcolo_ragnatela as spider
spider.extract_landmarks_4circles_4sectors(r"C:\Users\vince\Desktop\Progetto FVAB\shape_predictor_68_face_landmarks.dat",r'C:\Users\vince\Desktop\Progetto FVAB\merged_dataset\images')

# Classificatore
Usiamo le SVM

In [67]:
#Import svm model
from sklearn import svm
from sklearn import metrics
from numpy import loadtxt
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from joblib import dump, load
 

#Provare con 1000 immagini e cronometrare

import pandas as pd
from sklearn.model_selection import train_test_split

labels= loadtxt('data.csv',delimiter=',', usecols=(0,))
data= loadtxt('data.csv', delimiter=',',usecols=range(1,64))


X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, shuffle=False)

svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

dump(svclassifier, 'filename.joblib') 


y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred,target_names=['female','male']))



[[  6   3]
 [  2 117]]
              precision    recall  f1-score   support

      female       0.75      0.67      0.71         9
        male       0.97      0.98      0.98       119

    accuracy                           0.96       128
   macro avg       0.86      0.82      0.84       128
weighted avg       0.96      0.96      0.96       128

